# Methodology

## 1. **Data Acquisition and Preprocessing**  
   - Source of data: scFoundation repository
   - datasets: CCLE, GDSC, 
   - Adaptations:
       - applied zero padding and normalization to gene expression data similar to scF preprocessing
       - difference to scFoundation: no positional embedding creation (look embedding.py file in repo for reference)
       - Adjusted dataset size at leave drug to prevent RAM overflow on kaggle (90000 datapoints)  
   - Data format: Describe how drug response and embeddings were structured
       - all data stems from the scFoundation repository 
        - gene expression data:
            - bulk level gene expression data for 561 cancer cell lines over 697 genes.
        - GDSC_IC50:
            - IC50 values for drug/cellline combinations
        - Drug info file:
            - Mapping of GDSC IDs with PubCHEM IDs
        - Drug graphs:
            - folder containing a file for each drug, named by pubCHEM ID (223 Files)
            - files contain graph representations:
                - feature matrix (atoms)
                - adjacency list (bonds)
                - degree list(number of bound for each atom, indicating importance of the atom))
        - gene list:
            - genes to include in computation of the embedding, initialize those with value 0 which dont exist in expression data
        - scFoundation embeddings:
            - numpy array containing the precomputed embeddings. 

## 2. **Creation of the Embeddings**

### Embedding Models:
- scFoundation: Pre-trained embeddings from the scFoundation repository
- scGPT: Pre-trained scGPT model

### Embedding Extraction Process:
- Input data (gene expression data)
- retrieved an embedding for each cellline
- Preprocessing steps before feeding into the embedding models:
    - zero padding for not existing genes from given gene list
    - normalization using log1p
- retrieving embeddings from scGPTs embed_data method
- Embedding Shape and Size: Comparison of scGPT (512) and scFoundation (768) embeddings 

## 3. **Model Adaptation and Training**  
   - **Baseline Model**: Standard DeepCDR without additional embeddings  
   - **scFoundation Model**: DeepCDR with scFoundation embeddings  
   - **scGPT Model**: DeepCDR with scGPT embeddings  
   - Code modifications: Adapted DeepCDR to run in kaggle with our embeddings  

## 4. **Evaluation Metrics and Analysis**  
   - **Primary metric**: Pearson correlation coefficient  
   - **Evaluation methods**:  
     - Performance by drug  
     - Performance by cell line  
     - Performance by cancer type
     - Training stability
     - leave-one-drug-out (20 sampled drugs)
   - **Comparisons**: Benchmarked scGPT against scFoundation and baseline  

## 5. **Computational Setup**  
   - Hardware/software environment: kaggle  
   - Training constraints (e.g., runtime, GPU limitations)
       - main constraint: RAM (deepCDR implementation is memory heavy)   


# Creation of scGPT embeddings as cell line representation

In [ ]:
!pip install hickle

import tensorflow as tf
import keras
import hickle

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 108.0/108.0 kB 3.1 MB/s eta 0:00:00
2.17.1 3.5.0 5.0.3


## Install deps

In [1]:
!pip install scanpy
!pip install scib
!pip install scgpt

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 33.7 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 142.7/142.7 kB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.9/56.9 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.9/294.9 kB 18.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.8/88.8 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.7/52.7 kB 2.8 MB/s eta 0:00:00
  Attempting uninstall: seaborn
    Found existing installation: seaborn 0.12.2
    Uninstalling seaborn-0.12.2:
      Successfully uninstalled seaborn-0.12.2
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.6/84.6 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 45.7 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.8/188.8 kB 11.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
INFO: pip is looking at multiple vers

In [2]:
from pathlib import Path
import warnings

import scanpy as sc
import scib
import numpy as np
import sys

sys.path.insert(0, "../")

import scgpt as scg
import matplotlib.pyplot as plt
import anndata

plt.style.context('default')
warnings.simplefilter("ignore", ResourceWarning)



/usr/local/lib/python3.10/dist-packages/scgpt/model/model.py:21: UserWarning: flash_attn is not installed
  warnings.warn("flash_attn is not installed")
/usr/local/lib/python3.10/dist-packages/scgpt/model/multiomic_model.py:19: UserWarning: flash_attn is not installed
  warnings.warn("flash_attn is not installed")


## Download scGPT-human model checkpoint (zero shot tutorial)

In [3]:
import os
import sys
# CODE TO DOWNLOAD SCGPT-Human checkpoint

model_dir = "/kaggle/working/scGPT_human"

print("Downloading data and model ckpt...")
!pip install -q -U gdown
import gdown

print("Downloading model ckpt...")
if not os.path.exists(model_dir):
    !mkdir -p $model_dir
    gdown.download_folder(
        "https://drive.google.com/drive/folders/1oWh_-ZRdhtoGQ2Fw24HP41FgLoomVo-y",
        output=model_dir,
    )


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


## Preprocess gene expression data
### Add unknown genes as zero padding, normalize expression values

Code taken from scFoundation embedding generation preprocessing.
Difference is scFoundation creates positional embeddings first when creating the full embeddings, what we for scGPT don't do.

Displaying the created expression data dataframe after.

In [4]:
def main_gene_selection(X_df, gene_list):
    """
    Describe:
        rebuild the input adata to select target genes encode protein 
    Parameters:
        adata->`~anndata.AnnData` object: adata with var index_name by gene symbol
        gene_list->list: wanted target gene 
    Returns:
        adata_new->`~anndata.AnnData` object
        to_fill_columns->list: zero padding gene
    """
    to_fill_columns = list(set(gene_list) - set(X_df.columns))
    padding_df = pd.DataFrame(np.zeros((X_df.shape[0], len(to_fill_columns))), 
                              columns=to_fill_columns, 
                              index=X_df.index)
    X_df = pd.DataFrame(np.concatenate([df.values for df in [X_df, padding_df]], axis=1), 
                        index=X_df.index, 
                        columns=list(X_df.columns) + list(padding_df.columns))
    X_df = X_df[gene_list]
    
    var = pd.DataFrame(index=X_df.columns)
    var['mask'] = [1 if i in to_fill_columns else 0 for i in list(var.index)]
    return X_df, to_fill_columns

In [ ]:
import pandas as pd
import torch
import numpy as np
gexpr_feature = pd.read_csv("..data/CCLE/genomic-expression/genomic_expression_561celllines_697genes_demap_features.csv",index_col=0)
gene_list_df = pd.read_csv('../data/OS_scRNA_gene_index.19264.tsv', header=0, delimiter='\t')
gene_list = list(gene_list_df['gene_name'])
gexpr_feature, to_fill_columns = main_gene_selection(gexpr_feature,gene_list)
data = gexpr_feature.to_numpy(dtype=np.float32)
data = data/data.sum(1,keepdims=True)*10000
data = np.log1p(data)
gexpr_feature = pd.DataFrame(data,index=gexpr_feature.index,columns=gexpr_feature.columns)

gexpr_feature

#adata = sc.read_csv("/kaggle/input/genomic-expression/genomic_expression_561celllines_697genes_demap_features.csv")
#adata.obs

A1BG      A1CF  A2M  A2ML1  A3GALT2  A4GALT  A4GNT  AAAS  AACS  \
ACH-000828   0.0  0.000000  0.0    0.0      0.0     0.0    0.0   0.0   0.0   
ACH-000568   0.0  0.122203  0.0    0.0      0.0     0.0    0.0   0.0   0.0   
ACH-000560   0.0  0.152391  0.0    0.0      0.0     0.0    0.0   0.0   0.0   
ACH-000561   0.0  0.160657  0.0    0.0      0.0     0.0    0.0   0.0   0.0   
ACH-000562   0.0  0.161598  0.0    0.0      0.0     0.0    0.0   0.0   0.0   
...          ...       ...  ...    ...      ...     ...    ...   ...   ...   
ACH-000242   0.0  0.108701  0.0    0.0      0.0     0.0    0.0   0.0   0.0   
ACH-000245   0.0  0.112987  0.0    0.0      0.0     0.0    0.0   0.0   0.0   
ACH-000244   0.0  0.000000  0.0    0.0      0.0     0.0    0.0   0.0   0.0   
ACH-000247   0.0  2.321918  0.0    0.0      0.0     0.0    0.0   0.0   0.0   
ACH-000369   0.0  0.000000  0.0    0.0      0.0     0.0    0.0   0.0   0.0   

            AADAC  ...  ZWILCH  ZWINT  ZXDA  ZXDB  ZXDC  ZYG11A  ZYG11B  ZYX  \
ACH-000828    0.0  ...     0.0    0.0   0.0   0.0   0.0     0.0     0.0  0.0   
ACH-000568    0.0  ...     0.0    0.0   0.0   0.0   0.0     0.0     0.0  0.0   
ACH-000560    0.0  ...     0.0    0.0   0.0   0.0   0.0     0.0     0.0  0.0   
ACH-000561    0.0  ...     0.0    0.0   0.0   0.0   0.0     0.0     0.0  0.0   
ACH-000562    0.0  ...     0.0    0.0   0.0   0.0   0.0     0.0     0.0  0.0   
...           ...  ...     ...    ...   ...   ...   ...     ...     ...  ...   
ACH-000242    0.0  ...     0.0    0.0   0.0   0.0   0.0     0.0     0.0  0.0   
ACH-000245    0.0  ...     0.0    0.0   0.0   0.0   0.0     0.0     0.0  0.0   
ACH-000244    0.0  ...     0.0    0.0   0.0   0.0   0.0     0.0     0.0  0.0   
ACH-000247    0.0  ...     0.0    0.0   0.0   0.0   0.0     0.0     0.0  0.0   
ACH-000369    0.0  ...     0.0    0.0   0.0   0.0   0.0     0.0     0.0  0.0   

            ZZEF1  ZZZ3  
ACH-000828    0.0   0.0  
ACH-000568    0.0   0.0  
ACH-000560    0.0   0.0  
ACH-000561    0.0   0.0  
ACH-000562    0.0   0.0  
...           ...   ...  
ACH-000242    0.0   0.0  
ACH-000245    0.0   0.0  
ACH-000244    0.0   0.0  
ACH-000247    0.0   0.0  
ACH-000369    0.0   0.0  

[561 rows x 19264 columns]

### Create AnnData object 

In [8]:
adata = anndata.AnnData(gexpr_feature)
adata.X

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


array([[0.       , 0.       , 0.       , ..., 0.       , 0.       ,
        0.       ],
       [0.       , 0.1222032, 0.       , ..., 0.       , 0.       ,
        0.       ],
       [0.       , 0.1523914, 0.       , ..., 0.       , 0.       ,
        0.       ],
       ...,
       [0.       , 0.       , 0.       , ..., 0.       , 0.       ,
        0.       ],
       [0.       , 2.3219175, 0.       , ..., 0.       , 0.       ,
        0.       ],
       [0.       , 0.       , 0.       , ..., 0.       , 0.       ,
        0.       ]], dtype=float32)

## Create scGPT embeddings

In [9]:
import os
model_dir = '/kaggle/working/scGPT_human/'

gene_col = "gene_name"
adata.var[gene_col] = adata.var.index
adata.obs["cell_line_id"] = adata.obs.index
embed_adata = scg.tasks.embed_data(
    adata,
    model_dir,
    #device = torch.device("cpu"),
    use_fast_transformer=False,
    gene_col=gene_col,
    batch_size=64,
)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


scGPT - INFO - match 19088/19264 genes in vocabulary of size 60697.


Embedding cells: 100%|██████████| 9/9 [00:05<00:00,  1.74it/s]
/usr/local/lib/python3.10/dist-packages/scgpt/tasks/cell_emb.py:279: ImplicitModificationWarning: Setting element `.obsm['X_scGPT']` of view, initializing view as actual.
  adata.obsm["X_scGPT"] = cell_embeddings


## Load and Display created scGPT embeddings

In [10]:
embedding_dir = "/kaggle/working/scGPT_embeddings"
if not os.path.exists(embedding_dir):
    !mkdir -p $embedding_dir
    
np.save(embedding_dir+"/scGPT_embedding.npy", embed_adata.obsm["X_scGPT"])

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [11]:
scGPT_emb = np.load("/kaggle/working/scGPT_embeddings/scGPT_embedding.npy")
scFoundation_emb = np.load("/kaggle/input/scfoundation/50M-0.1B-res_embedding.npy")

In [12]:
scf_df = pd.DataFrame(scFoundation_emb,index=gexpr_feature.index)
scg_df = pd.DataFrame(scGPT_emb, index = gexpr_feature.index)

In [13]:
scg_df

0         1         2         3         4         5    \
ACH-000828 -0.082104  0.040278  0.009901  0.041053 -0.038335 -0.033598   
ACH-000568 -0.073366  0.043746  0.005449  0.024601 -0.018280 -0.037672   
ACH-000560 -0.081323  0.018478  0.012042  0.028158 -0.026500 -0.009306   
ACH-000561 -0.082685  0.004444 -0.014467  0.029661 -0.032018 -0.026173   
ACH-000562 -0.105183  0.023034 -0.000140  0.048637 -0.034725 -0.012403   
...              ...       ...       ...       ...       ...       ...   
ACH-000242 -0.088996  0.021753  0.027686  0.053887 -0.036950 -0.029444   
ACH-000245 -0.067055 -0.021289 -0.043369  0.011949 -0.019980 -0.011927   
ACH-000244 -0.089535  0.008861 -0.010587 -0.000924 -0.012846 -0.063334   
ACH-000247 -0.071617  0.036104  0.028237  0.034242 -0.023732 -0.041984   
ACH-000369 -0.063434 -0.003011 -0.023220 -0.023219  0.004829 -0.024853   

                 6         7         8         9    ...       502       503  \
ACH-000828  0.007201  0.000245  0.048542  0.011302  ...  0.007792  0.041356   
ACH-000568  0.000645  0.006192  0.058254  0.000021  ...  0.013795  0.027767   
ACH-000560  0.007690  0.002898  0.030013  0.015974  ...  0.011411  0.055345   
ACH-000561  0.019165  0.000738  0.047491  0.034335  ...  0.017459  0.045706   
ACH-000562  0.027275 -0.002993  0.062146  0.007853  ...  0.018950  0.036000   
...              ...       ...       ...       ...  ...       ...       ...   
ACH-000242  0.021938  0.014165  0.055734  0.022737  ...  0.006410  0.045877   
ACH-000245  0.063880 -0.023276  0.009161  0.025342  ...  0.034479  0.038948   
ACH-000244  0.013665 -0.027753  0.084460  0.000709  ...  0.000763  0.027737   
ACH-000247  0.028764  0.032950  0.054692  0.042018  ... -0.014426  0.033467   
ACH-000369  0.050861 -0.003487  0.039178  0.010710  ...  0.016768  0.018659   

                 504       505       506       507       508       509  \
ACH-000828  0.055791 -0.048203  0.056692 -0.042192  0.051700 -0.052856   
ACH-000568  0.086038 -0.036307  0.042305 -0.043068  0.057922 -0.054698   
ACH-000560  0.055309 -0.026276  0.046953 -0.039326  0.055355 -0.051241   
ACH-000561  0.056834 -0.046998  0.062713 -0.065552  0.056223 -0.028112   
ACH-000562  0.042228 -0.064383  0.035648 -0.048296  0.049161 -0.048575   
...              ...       ...       ...       ...       ...       ...   
ACH-000242  0.073990 -0.038238  0.067770 -0.041648  0.058308 -0.059080   
ACH-000245  0.035806 -0.001045  0.057679 -0.020279  0.041904 -0.072545   
ACH-000244  0.063659 -0.039069  0.004588 -0.060842  0.054772 -0.031580   
ACH-000247  0.083433 -0.052538  0.026625 -0.032265  0.059627 -0.070790   
ACH-000369  0.036235 -0.039042  0.022269 -0.041629  0.066489 -0.054946   

                 510       511  
ACH-000828 -0.017793 -0.008415  
ACH-000568 -0.033704 -0.013930  
ACH-000560 -0.013679 -0.028595  
ACH-000561 -0.017509 -0.003454  
ACH-000562 -0.023881  0.007488  
...              ...       ...  
ACH-000242 -0.024520 -0.005858  
ACH-000245  0.003094 -0.047685  
ACH-000244 -0.004779  0.027489  
ACH-000247 -0.029859 -0.011956  
ACH-000369  0.023790 -0.023719  

[561 rows x 512 columns]

In [14]:
scf_df

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


0         1         2         3         4         5    \
ACH-000828  0.134085  2.426087 -0.173301  0.562729  0.037409 -0.221054   
ACH-000568  0.158275  2.335788 -0.171646  0.422805  0.011783 -0.096568   
ACH-000560  0.248130  2.515146  0.017260  0.511272 -0.093265 -0.167926   
ACH-000561  0.248951  2.658738  0.001120  0.480017  0.104127 -0.210335   
ACH-000562  0.286745  2.590119 -0.047205  0.475028  0.140947 -0.122147   
...              ...       ...       ...       ...       ...       ...   
ACH-000242  0.316036  2.481954 -0.078085  0.517848 -0.223953 -0.200417   
ACH-000245  0.650149  2.833161  0.100378  0.498989 -0.353675 -0.208170   
ACH-000244  0.191077  2.350697  0.131426  0.551015 -0.123706 -0.107079   
ACH-000247  0.400594  2.645751 -0.014980  0.495615  0.068938 -0.226251   
ACH-000369  0.730871  2.596743  0.048081  0.526470  0.070145 -0.135396   

                 6         7         8         9    ...       758       759  \
ACH-000828  0.786906  0.261464  0.093412  1.737628  ...  1.297960  2.949629   
ACH-000568  0.852044  0.315544  0.314160  1.760733  ...  1.405602  3.063999   
ACH-000560  0.864610  0.538356  0.095087  1.736143  ...  1.258035  3.008534   
ACH-000561  0.787900  0.466651  0.139311  1.727135  ...  1.242655  2.923209   
ACH-000562  0.719855  0.465842  0.726302  1.662443  ...  1.265865  2.949314   
...              ...       ...       ...       ...  ...       ...       ...   
ACH-000242  0.591604  0.396492  0.188908  1.792275  ...  1.335042  3.174084   
ACH-000245  0.410629  0.691112  0.048696  1.878688  ...  1.629580  3.021252   
ACH-000244  0.675239  0.328413  0.403786  1.906664  ...  1.247062  3.229273   
ACH-000247  0.591276  0.674510  0.156106  1.604940  ...  1.344422  3.063221   
ACH-000369  0.414512  0.728146  0.042955  1.782846  ...  1.796818  3.147300   

                 760       761       762       763       764       765  \
ACH-000828 -0.360722  2.418570  2.535873 -0.205821  2.498439  1.263877   
ACH-000568 -0.236939  2.313529  2.440657 -0.496516  2.580294  1.202098   
ACH-000560 -0.403137  2.513517  2.356425 -0.214048  2.604833  1.251498   
ACH-000561 -0.454310  2.428309  2.338891 -0.166368  2.514753  1.406705   
ACH-000562 -0.111720  2.394945  2.338046 -0.314805  2.441266  1.323736   
...              ...       ...       ...       ...       ...       ...   
ACH-000242 -0.181894  2.496228  2.302829 -0.382567  2.538488  1.209668   
ACH-000245 -0.300246  2.611536  2.109222 -0.216214  2.654089  1.358388   
ACH-000244 -0.458808  2.362687  2.348300 -0.233766  2.875983  1.384020   
ACH-000247 -0.337893  2.680469  2.333794 -0.236084  2.352339  1.292253   
ACH-000369 -0.458136  2.710165  2.171620 -0.550169  2.740461  1.338861   

                 766       767  
ACH-000828  0.229625 -0.288320  
ACH-000568  0.222224 -0.090608  
ACH-000560  0.385761 -0.241133  
ACH-000561  0.262775 -0.184308  
ACH-000562  0.180582 -0.356424  
...              ...       ...  
ACH-000242  0.224096 -0.443991  
ACH-000245  0.231034  0.193794  
ACH-000244  0.271026 -0.286424  
ACH-000247  0.305416 -0.505829  
ACH-000369  0.408906 -0.385322  

[561 rows x 768 columns]